<a href="https://colab.research.google.com/github/shubham-senani/Airbnb-Clone/blob/main/MA6_WahooFitness_Profiling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# =========================
# IMPORT LIBRARIES
# =========================
import pandas as pd
import numpy as np
import io
from google.colab import files

from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
import plotly.graph_objects as go

In [3]:
# =========================
# DATA INPUT OPTIONS
# =========================

print("Choose how you want to provide the dataset:")
print("1 → Upload CSV file (from your computer)")
print("2 → Load Google Sheet (as CSV)")

choice = input("Enter 1 or 2: ")

# -----------------------------------
# OPTION 1: Upload CSV manually
# -----------------------------------
if choice == "1":
    uploaded = files.upload()
    filename = list(uploaded.keys())[0]
    df = pd.read_csv(io.BytesIO(uploaded[filename]))

    print(f"File '{filename}' uploaded successfully.")

# -----------------------------------
# OPTION 2: Read Google Sheet as CSV
# -----------------------------------
elif choice == "2":
    #GOOGLE_SHEET_ID = "16L_uzD2MS-TOqon7lyL4onVkLvq2P2Y1"
    GOOGLE_SHEET_ID = input("Enter the Google Sheet ID: ").strip()
    DATA_URL = f"https://docs.google.com/uc?id={GOOGLE_SHEET_ID}&export=download"

    df = pd.read_csv(DATA_URL)
    print("Google Sheet loaded successfully.")

# -----------------------------------
# INVALID CHOICE
# -----------------------------------
else:
    raise ValueError("Invalid choice. Please rerun the cell and enter 1 or 2.")

# =========================
# DATA CHECK
# =========================
print("\nDataset shape:", df.shape)
print("Columns:", df.columns.tolist())

df.head()


Choose how you want to provide the dataset:
1 → Upload CSV file (from your computer)
2 → Load Google Sheet (as CSV)
Enter 1 or 2: 2
Enter the Google Sheet ID: 16L_uzD2MS-TOqon7lyL4onVkLvq2P2Y1
Google Sheet loaded successfully.

Dataset shape: (1313, 79)
Columns: ['CID', 'Q109_1', 'Class?', 'QS2', 'QS3', 'AGE', 'hqAge', 'hqSportsParticipate_1', 'hqSportsParticipate_2', 'hqSportsParticipate_3', 'hqSportsParticipate_4', 'hqSportsParticipate_98', 'hqSportsEver_1', 'hqSportsEver_2', 'hqSportsEver_3', 'hqSportsEver_4', 'hqSportsEver_5', 'hqSportsEver_6', 'hqSportsEver_7', 'hqSportsEver_8', 'hqSportsEver_9', 'hqSportsEver_10', 'hqSportsEver_11', 'hqSportsEver_12', 'hqSportsEver_98', 'hqNumberOfSports_1', 'hqRun', 'hqCycle', 'hqTri', 'hqRunCycleTri', 'hqSpend', 'QS6_1', 'QS6_2', 'QS6_3', 'QS6_4', 'QS6_5', 'QS6_6', 'QS6_7', 'QS6_8', 'QS6_9', 'QS6_10', 'QS6_11', 'QS6_98', 'hqTrack', 'hqGPS', 'QS7_1', 'QS7_2', 'QS7_3', 'QS7_4', 'QS7_5', 'QS7_6', 'QS7_7', 'QS7_8', 'QS7_9', 'QS8', 'hqIndoorTrainer'

,CID,Q109_1,Class?,QS2,QS3,AGE,hqAge,hqSportsParticipate_1,hqSportsParticipate_2,hqSportsParticipate_3,...,Q2_3,Q2_4,Q2_5,Q2_6,Q2_7,Q2_8,Q2_9,Q2_10,Q2_11,Q2_12
0,1,Sample,0,1,97,24,2,1.0,1.0,1.0,...,4,5,4,4,5,5,5,4,4,5
1,2,Sample,1,2,96,25,3,1.0,1.0,1.0,...,1,1,1,1,1,1,1,1,1,1
2,3,non-sample,2,1,92,29,3,1.0,1.0,1.0,...,4,5,5,1,5,4,1,1,5,4
3,4,Sample,0,1,90,31,4,1.0,1.0,1.0,...,4,3,4,2,5,4,2,4,5,4
4,5,Sample,0,1,90,31,4,1.0,1.0,1.0,...,3,3,3,3,3,5,5,5,5,5


In [4]:
# =========================
# DATA PARTITIONING
# =========================
# Create two datasets based on sampling condition

seg_data_non_sample = df[df["Q109_1"] == "non-sample"].copy()
seg_data_sample = df[df["Q109_1"] == "Sample"].copy()

# =========================
# USER SELECTION: Customer vs NON-Customer
# =========================

print("Select dataset for analysis:")
print("1 → Customer")
print("2 → Non-Customer")

user_choice = input("Enter 1 or 2: ").strip()

if user_choice == "1":
    seg_data = seg_data_sample
    data_flag = "customer"
elif user_choice == "2":
    seg_data = seg_data_non_sample
    data_flag = "non_customer"
else:
    raise ValueError("Invalid input. Please enter 1 or 2.")

print(f"Dataset selected: {data_flag}")

segmentation_columns = seg_data.columns[56:79]
segment_df = seg_data[segmentation_columns].copy()

Select dataset for analysis:
1 → Customer
2 → Non-Customer
Enter 1 or 2: 1
Dataset selected: customer


In [5]:
# =========================
# CLUSTER SIZE SUMMARY
# =========================

cluster_counts = seg_data["Class?"].value_counts().sort_index()

cluster_counts.head()

,count
Class?,
0,190
1,110
2,199
3,301


In [6]:
# =========================
# MEANS BY CLUSTER (SEGMENTATION QUESTIONS)
# =========================


cluster_means = (
    seg_data
    .groupby("Class?")[segmentation_columns]
    .mean()
    .T
)

cluster_means.columns = [f"cluster{i}" for i in cluster_means.columns]
cluster_means.head()

,cluster0,cluster1,cluster2,cluster3
Q1_1,2.684211,2.109091,3.376884,2.089701
Q1_2,3.042105,2.727273,3.648241,1.976744
Q1_3,2.310526,2.227273,2.341709,2.049834
Q1_4,2.468421,2.445455,2.572864,2.083056
Q1_5,2.657895,2.672727,2.678392,2.056478


In [7]:
# =========================
# PROFILE ANALYSIS
# =========================
# Profiling rules:
# - Binary indicators → value = 1 only, COUNT + % of CLUSTER
# - True categorical variables → COUNT only
# - Percentages for binaries are calculated using cluster base
# - No mean-based summaries included

profile_tables = []

# -------------------------------------------------
# Helper: categorical variables (COUNTS ONLY)
# -------------------------------------------------
def categorical_profile_counts(df, var, cluster_col="Class?"):
    """
    Returns counts only for all categories
    of a categorical variable by cluster.
    """
    count_tab = pd.crosstab(df[var], df[cluster_col])
    count_tab.index = [f"{var}: {idx}" for idx in count_tab.index]
    return count_tab

# -------------------------------------------------
# Helper: binary variables (value == 1, COUNT + %)
# -------------------------------------------------
def binary_profile_cluster_pct(df, var, cluster_col="Class?"):
    """
    Returns count and within-cluster percentage
    for binary indicators where value == 1 only.
    """
    # Cluster base sizes
    cluster_base = df[cluster_col].value_counts().sort_index()

    # Keep only value == 1
    df_ones = df[df[var] == 1]

    # Count of 1s by cluster
    count_tab = pd.crosstab(df_ones[var], df_ones[cluster_col])

    # Ensure all clusters are present
    count_tab = count_tab.reindex(columns=cluster_base.index, fill_value=0)

    # Percentage of cluster total
    pct_tab = count_tab.div(cluster_base, axis=1) * 100

    # Combine count and percentage
    combined = count_tab.astype(str) + " (" + pct_tab.round(1).astype(str) + "%)"
    combined.index = [f"{var}: Yes (1)"]

    return combined

# -------------------------------------------------
# TRUE CATEGORICAL VARIABLES (COUNTS ONLY)
# -------------------------------------------------
categorical_vars = [
    "QS2",      # Gender
    "hqAge",    # Age group
    "hqSpend"   # Spend category
]

for var in categorical_vars:
    profile_tables.append(categorical_profile_counts(seg_data, var))

# -------------------------------------------------
# BEHAVIOR / FEATURE INDICATORS (BINARY)
# -------------------------------------------------
binary_vars = [
    "hqRun",
    "hqCycle",
    "hqTri",
    "hqRunCycleTri",
    "hqGPS",
    "hqTrack"
]

for var in binary_vars:
    profile_tables.append(binary_profile_cluster_pct(seg_data, var))

# -------------------------------------------------
# DEVICE & EQUIPMENT OWNERSHIP (QS6, QS7)
# -------------------------------------------------
# All questions retained, only value == 1 reported
# Percentages are within-cluster

qs6_cols = [c for c in seg_data.columns if c.startswith("QS6_")]
qs7_cols = [c for c in seg_data.columns if c.startswith("QS7_")]

for battery, label in [(qs6_cols, "QS6"), (qs7_cols, "QS7")]:
    for var in battery:
        tab = binary_profile_cluster_pct(seg_data, var)
        tab.index = [f"{label} Ownership: {var} = 1"]
        profile_tables.append(tab)

# -------------------------------------------------
# COMBINE PROFILE OUTPUTS
# -------------------------------------------------
# Final table structure:
# - Categorical variables → counts
# - Binary indicators → count + % of cluster

profile_summary = pd.concat(profile_tables)
profile_summary.columns = [f"cluster{i}" for i in profile_summary.columns]

In [8]:
# =========================
# EXPORTS To FILE
# =========================

segmented_data_file = f"cluster_segmented_data_{data_flag}.csv"
cluster_means_file = f"cluster_segmentation_means_{data_flag}.csv"
profile_summary_file = f"cluster_profile_summary_{data_flag}.csv"

seg_data.to_csv(segmented_data_file, index=True)
cluster_means.to_csv(cluster_means_file, index=True)
profile_summary.to_csv(profile_summary_file, index=True)

files.download(segmented_data_file)
files.download(cluster_means_file)
files.download(profile_summary_file)

# ============================================================
# END OF SCRIPT
# ============================================================

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>